# 네이버 기사 크롤링
### - 크롤링을 오래 돌리면 막힐 수 있기 때문에 월별로 끊어서 크롤링 진행함

In [20]:

from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import numpy as np
import time
import pandas as pd
import urllib
from tqdm.notebook import tqdm
import re


def get_url(query, start_date, end_date, start_page):
    # 네이버 뉴스검색에서 검색할 뉴스의 카테고리, 시간 범위 옵션을 설정했을 때의 url을 생성
    base = "https://search.naver.com/search.naver?where=news&query={search_words}"
    ds = start_date
    de = end_date
    start = start_page
    url = base + "&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=" + ds + "&de=" + de + "&start=" + start
    request_url = Request(url.format(search_words = urllib.parse.quote(query)))
    return request_url


def set_selenium_opt():
    # 크롤링을 하기 위한 selenlium 크롬 드라이버의 설정값
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 브라우저를 표시하지 않음
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--dns-prefetch-disable")
    chrome_options.add_argument("--disable-logging")
    chrome_options.add_argument("--disable-background-networking")
    chrome_options.add_argument("--disable-default-apps")
    driver = webdriver.Chrome(options=chrome_options)
    return driver


def get_page_html(driver, url):
    # 해당 url 페이지의 전체 html 추출
    driver.get(url)
    page_html = driver.page_source
    # print(page_html)
    return page_html


def filter_naver_links(links):
    # 추출한 링크들 중 네이버 뉴스 링크들만 선별
    return [link for link in links if 'news.naver.com' in link]


def do_link_crawling(driver, url):
    # 특정 키워드, 범위 안의 네이버 기사들을 크롤링
    
    link_list = []

    # 해당 url 페이지에서 추출한 html이 제대로 존재하는지 여부를  확인
    while True:
        html_content = get_page_html(driver, url.full_url)

        if html_content:
            break
        print("Warning!!!")
        time.sleep(1)
        
    # BeautifulSoup을 사용하여 페이지 파싱
    soup = BeautifulSoup(html_content, "html.parser")
    group = soup.find("ul", "list_news")
    
    if not group:
        return []
    
    bx = group.find_all("li", "bx")
    for li_tag in bx:
        a_tags = li_tag.find_all("a")
        for a_tag in a_tags:
            link_list.append(a_tag["href"])
    return link_list


def do_article_crawling(driver, url):
    
    date = []
    title = []
    text = []
    
    for row in url:
        # 태그 형식이 하나라도 다르다면 NAN값을 집어넣어줌
        try:
            driver.get(row)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h2")))
        
            date.append(driver.find_element(By.CLASS_NAME, "media_end_head_info_datestamp_time._ARTICLE_DATE_TIME").get_attribute("data-date-time"))
            title.append(driver.find_element(By.ID, "title_area").get_attribute("innerText"))
            text.append(driver.find_element(By.ID, "dic_area").get_attribute("innerText"))
        except NoSuchElementException:
            date.append(np.NAN)
            title.append(np.NAN)
            text.append(np.NAN)
            
    
    return pd.DataFrame({"date" : date, "title" : title, "text" : text})

def get_refined_article(article):
    
    # 중복된 기사 삭제
    article = article.drop_duplicates(["date", "text"], keep = "first")
    
    # 기사 내용이 없는 데이터의 경우 삭제처리
    article[article["text"] == "nan"] = np.nan
    article.dropna(subset=["text"],how="any", axis = 0, inplace=True)

    # 제목에서 문자, 스페이스만 남기고 제거
    article["title"] = article["title"].apply(lambda x : re.sub("[^A-Za-z0-9가-힣]", " ", x))
    article["title"] = article["title"].apply(lambda x : re.sub(" +", " ", x))

    # 내용에서 문자, 스페이스만 남기고 제거
    article["text"] = article["text"].apply(lambda x : re.sub("[^A-Za-z0-9가-힣]", " ", x))
    article["text"] = article["text"].apply(lambda x : re.sub(" +", " ", x))
    
    return article


def get_stopWords():
    stop_words = open("../../data/koreanStopwords.txt", "r")
    lines = stop_words.readlines()
    stop_word = []
    for line in lines:
        line = line.replace("\n", "")
        stop_word.append(line)
    stop_words.close()
    return stop_word


def get_tokenized_article(refined_article):
    stop_words = get_stopWords()
    m = Mecab()
    tokenized = []

    for row in refined_article:
        inner = []
        text = m.morphs(row)
        
        for word in text:
            
            
            if (len(word) == 1) or (word in stop_words):
                pass
            else:
                inner.append(word)

        tokenized.append(inner)
    
    return tokenized


def main():
    link_list = []
    news_info_list = []

    start_date = datetime.strptime("2023.12.01", "%Y.%m.%d")
    end_date = datetime.strptime("2023.12.01", "%Y.%m.%d")
    current_date = start_date
    
    formatted_date = str(current_date.strftime("%Y.%m.%d"))
    
    start_page = 1

    driver = set_selenium_opt()
        
    while current_date <= end_date:
        formatted_date = str(current_date.strftime("%Y.%m.%d"))

        for start_page in range(1, 101, 10):
            url = get_url("전기차", str(start_date.strftime("%Y.%m.%d")), formatted_date, str(start_page))
            link = do_link_crawling(driver, url)
            link_list.extend(link)
        current_date += timedelta(days=3)
        
    
    naver_link_list = filter_naver_links(link_list)
        
    news_info_list = do_article_crawling(driver, naver_link_list)
        
    refined_news_info = get_refined_article(news_info_list)
    
    refined_news_info["title_tokenized"] = get_tokenized_article(refined_news_info["title"])
    refined_news_info["text_tokenized"] = get_tokenized_article(refined_news_info["text"])
    
    # print(naver_link_list)
    #df = pd.DataFrame(naver_link_list)
    #df.to_csv("~/amr_ws/eda/data/eda_project/eda_project_link.csv")

    driver.close()
    driver.quit()
    
    return refined_news_info
    


In [21]:
a = main()["title_tokenized"]

/tmp/ipykernel_11967/3981952102.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article[article["text"] == "nan"] = np.nan
/tmp/ipykernel_11967/3981952102.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article[article["text"] == "nan"] = np.nan
/tmp/ipykernel_11967/3981952102.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article.dropna(subset=["text"],how="an

In [10]:
dd = []

if dd:
    print("a")

In [35]:
a["text"]

0     반도체 수급난 여파 지속 내수 판매량은 11.4 감소 해외 판매도 18.4 줄어 서...
1      경향신문 현대차 아이오닉 5. 현대차 제공 국내 대표 완성차 업체인 현대차가 글로...
2     현대차 11월 판매 실적 현대자동차 더 뉴 그랜저 . 사진 현대차 제공. 현대자동차...
3      아산 뉴스1 장수영 기자 반도체 수급난으로 현대자동차 등 국내 완성차업체가 셧다운...
4     31만 2 602대 판매 전년비 17.1 감소 내수 6만 2 071대... 11.4...
                            ...                        
72     중남미 중동 아프리카서 점유율 하락세 반도체 공급난 중국 업체 공세 공장 중단 등...
73    반도체특별법이 9개월간의 표류 끝에 국회 상임위원회 소위원회 문턱을 넘었다. 국회 ...
74     경향신문 디스플레이 산업에서 일하던 남성 노동자가 병을 안고 태어난 아이에 대해 ...
75     이데일리 박정수 기자 차량용 시스템 반도체 기업 트루윈 105550 이 토지자산 ...
76    사진 트루윈 제공 파이낸셜뉴스 차량용 시스템 반도체 기업 트루윈이 토지자산 재평가를...
Name: text, Length: 72, dtype: object

### - 크롤링 진행한 월별 데이터들을 한 번에 불러와 하나로 합치기

In [2]:
from glob import glob
import pandas as pd
file_list = glob("../../data/CES/*")

df = pd.DataFrame()

for file in file_list:
    df = pd.concat([df, pd.read_csv(file)])

df.drop(labels="Unnamed: 0", axis=1, inplace=True)
df.sort_values("date", inplace=True)
df.set_index("date", inplace=True)

df

,title,text,prediction,score
date,,,,
2023-12-01 00:11:03,지역 컨벤션센터 수익성 넘어 경제 사회 문화 파급효과 따져야 MICE,['강릉시 주최 강릉 마이스도시 포럼 김봉석 교수 컨벤션센터 역할 기조강연 기업활동...,중립,0.537118
2023-12-01 00:21:01,K마이스 지속성장 위해 총괄법 제정하고 ESG 기회 삼아야 MICE,['전시산업전 마이스 엑스포 마이스인쇼 릴레이 개최 전문가들 산업영역 타깃시장 재정...,중립,0.513209
2023-12-01 07:15:00,헬스가전 업계 국내 넘어 CES 2024 서 기술 대결 펼친다,['바디프랜드 세라젬 CES 혁신상 수상하며 전초전 로보워킹 앞세운 바디프랜드 신제...,중립,0.503080
2023-12-01 07:37:01,박은빈 계명대 대학원생 인셉션랩 CES 2024 에서 혁신상 수상,"['박은빈 인셉션랩 대표.', '계명대 제공 헤럴드경제 대구 김병진 기자 계명대는 ...",호재,0.698458
2023-12-01 09:07:01,롯데 기술 도입 및 전략 다변화 통해 혁신 가속화,['롯데마트 데일리안 임유정 기자 롯데는 AI 인공지능 메타버스 등 다양한 ICT ...,호재,0.661369
...,...,...,...,...
2024-01-23 16:39:03,한지형 오토노머스에이투지 대표 2024 CES 모빌리티 리뷰,['ICT 전문가들이 전하는 CES2024 리뷰 인사이트 콘서트가 전자신문 주최로 ...,중립,1.000000
2024-01-23 17:32:03,초대석 CES 참석과 솔로몬의 선택 시민 위해 변화하는 성남,['출연 신상진 성남시장 경기 성남시가 올해 새로운 성남의 장을 열겠다는 포부를 밝...,호재,0.624096
2024-01-23 17:49:10,원컨덕터 2024 CES 참가 TSID 운영체계의 장애인용 키오스크 선봬,['원컨덕터가 시흥산업진흥원의 지원으로 2024년 CES 국제전자제품박람회 에 참가...,호재,0.810550


In [2]:
file_list

['../../data/CES/predicted_CES_2024_01.csv',
 '../../data/CES/predicted_CES_2023_12.csv']

### - 데이터 프레임 구조 편짐

In [2]:
df.drop(labels="Unnamed: 0", axis=1, inplace=True)
df.sort_values("date", inplace=True)
df.set_index("date", inplace=True)
df


,title,text
date,,
2023-01-01 03:00:00,투데이 반도체 기술로 만드는 한국형 스마트팜,UFO칼럼 사물인터넷 IoT 빅데이터 인공지능 AI 로봇 등으로 상징되는 4차 산...
2023-01-01 05:00:00,신년인터뷰 김태흠 충남지사 10월 독일서 반도체 전기차 투자유치 설명회,혁신도시 완성 위해 수도권 공공기관 유치 사회안전망 구축 최선 김태흠 충남지사가 ...
2023-01-01 06:00:00,신년사 김진태 강원지사 도민 중심 특별자치도 실현,예산보다 규제 풀 수 있는 권한 가진 분권 강조 올해 빚 10 더 줄이고 민생 일자...
2023-01-01 06:01:38,반도체 코리아 생각하는 메모리 상상치 못 한 일 벌어질 것 비밀병기 준비하는 삼성전자,70년 된 컴퓨팅 생태계에 지각변동 진행 중 데이터 병목현상에 발목 잡힌 반도체 혁...
2023-01-01 06:01:44,2023 증시전망 올해 코스피 반도체 이차전지 바이오 맑음,조선비즈 17개 증권사 리서치센터 설문조사 전체 증권사 중 8곳이 이차전지 업종 유...
...,...,...
2023-12-31 17:52:05,갑진년 용틀임 기대할 ETF는 반도체 AI,증권 운용사 16곳 추천 ETF 국내 상장 ETF 29개 꼽아 반도체 섹터 7개로 ...
2023-12-31 18:01:07,새해 반도체 시장 패권 3C 에 달렸다,삼성 SK 인텔 합종연횡 AI칩 효율적 생산하는 칩렛 데이터 처리속도 높이는 CXL...
2023-12-31 19:15:01,경상북도 올해 기업 투자유치 14조원 달성,경북도 제공 경상북도가 올해 역대 최대 규모인 14조1천802억원의 기업 투자유치 ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6842 entries, 2021-01-01 00:00:00 to 2021-12-31 15:50:01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6842 non-null   object
 1   text    6842 non-null   object
dtypes: object(2)
memory usage: 160.4+ KB


In [5]:

# df.to_csv("../../data/eda_project/2023_doosan_predicted.csv", sep = ",")
df.to_csv("../../data/CES/2023_CES_predicted.csv", sep = ",")
